In [1]:
import sys
sys.path.append('d:/flagellar/code/core')
sys.path.append('/flagellar/code/core/')
sys.path.append('/kaggle/input/my-flagellar-library/')
import flg_support as fls
import flg_runner
import importlib
import flg_model
import numpy as np
import matplotlib.pyplot as plt
import time
import copy
import os
import itertools
import glob

fast_mode = (fls.env=='local')
clear_data = True
local_mode = False
which_set = 1

MainProcess


In [2]:
# # Prepare datasets
if which_set == 0:
    dataset_str = ''
else:
    dataset_str = '-'+str(which_set+1)
fls.download_kaggle_dataset('jeroencottaar/byu-many-models' +dataset_str+ '/', fls.result_dir + '/many_full/', skip_download=clear_data)
fls.download_kaggle_dataset('jeroencottaar/byu-many-models-abbreviated' +dataset_str+ '/', fls.result_dir + '/many_abbr/', skip_download=clear_data)

In [ ]:
#fls.profiling=True
import git 
repo = git.Repo(search_parent_directories=True)
git_commit_id = repo.head.object.hexsha
for i in itertools.count(start=1000*which_set):
    print('Starting seed ', i)
    r = flg_runner.baseline_runner(fast_mode=fast_mode, local_mode = local_mode)
    # if single_settings:
    #     r.use_missing_value = True
    #     r.modifier_dict['n_ensemble'].missing_value = 1
    #     r.modifier_dict['scale_percentile_value'].missing_value = 3.
    #     r.modifier_dict['scale_std_clip_value'].missing_value = 2.
    #     r.modifier_dict['n_epochs'].missing_value = 50
    #     r.modifier_dict['use_best_epoch'].missing_value = False
    #     r.modifier_dict['model_name'].missing_value = 'yolov9s'
    #     r.modifier_dict['fix_norm_bug'].missing_value = True
    #     r.modifier_dict['alternative_slice_selection'].missing_value = True
    #     r.modifier_dict['negative_slice_ratio'].missing_value = 0.1
    #     r.modifier_dict['ratio_of_motors_allowed'].missing_value = 0.5
    #r.use_missing_value = True
    # r.label = 'Test ensemble'
    # r.modifier_dict = dict()
    # r.modifier_dict['n_ensemble'] = flg_runner.pm(1, lambda r:(r.integers(4,5)).item(), flg_runner.yolo)    
    # r.base_model = flg_model.ThreeStepModelLabelBased()
    # r.base_model.train_data_selector.datasets = ['tom']
    # r.base_model.run_in_parallel = False
    # if fast_mode: r.base_model.step1Labels.n_epochs = 1
    r.git_commit_id = git_commit_id
    r.env = fls.env
    r.seed = i
    #r.base_model.train_data_selector.datasets = 'tom'
    #r.base_model.run_in_parallel = False
    if local_mode:
        assert not r.use_missing_value
        base_filename = r.label + '_' + str(r.seed) + 'L_' + git_commit_id[:8]   
    elif r.use_missing_value:
        base_filename = r.label + '_' + str(r.seed) + 'M_' + git_commit_id[:8]   
    else:
        base_filename = r.label + '_' + str(r.seed) + '_' + git_commit_id[:8]   
    if len(glob.glob(fls.result_dir + '/many_full/' + base_filename + '*'))>=1:
         print('Skipping')
         continue
    r.run()

    score_str = str(r.cv_score[2])[2:5]
    print(str(r.cv_score)[1:5])
    output_file_full = fls.result_dir + '/many_full/' + base_filename + '_' + score_str +' _f.pickle'
    output_file_abbr = fls.result_dir + '/many_abbr/' + base_filename + '_' + score_str +' _a.pickle'
    
    fls.dill_save(output_file_full, r)
    r.trained_model = 0
    fls.dill_save(output_file_abbr, r)
    if not fast_mode and fls.env=='vast':
       fls.upload_kaggle_dataset(fls.result_dir + '/many_full/')
       fls.upload_kaggle_dataset(fls.result_dir + '/many_abbr/')

Starting seed  1000
1910 21
400 21
{'seed': 1000, 'use_best_epoch': False, 'lr0': 0.0008021898222640447, 'cos_lr': True, 'mosaic': 1.0, 'concentration': 2, 'extra_data': True, 'trust_neg': 1, 'trust_extra': 2, 'model_name': 'yolov8s', 'use_pretrained_weights': False}
XXX 0.9523809523809523
